In [ ]:
"""Not used in paper. Used the hyperparameters trained in DeiT paper instead"""
import sys
import os
sys.path.append('../')

In [ ]:
from TrainFunctions.train_transformerresnet import train_transformerresnet
from functools import partial
from ray import tune
import ray
from ray.tune.schedulers import ASHAScheduler
from ray.tune import CLIReporter

In [ ]:
ray.shutdown()
os.environ['PYTHONPATH'] = '/Users/adithyagiri/Desktop/STS/Repo/'
ray.init(num_gpus=1, num_cpus=12)
config = {
        "lr": tune.choice([0.0003, 0.001, 0.003, 0.01, 0.03, 0.1]),
        "dropout_p": tune.choice([0.001, 0.003, 0.01, 0.03]),
        "momentum": tune.choice([0.8, 0.9, 0.99]),
        "weight_decay": tune.choice([0.0003, 0.001, 0.003, 0.01, 0.03, 0.1]),
    }
max_epochs = 64
scheduler = ASHAScheduler(
        metric="accuracy",
        mode="max",
        max_t=max_epochs,
        reduction_factor=4,
        grace_period=2,
)
reporter = CLIReporter(
        metric_columns=["loss", "accuracy", "training_iteration"])
result = tune.run(
    partial(train_transformerresnet, patch_size = 2, img_h = 32, img_w = 32, d_model = 32, tuning_mode = True),
    config = config,
    num_samples = 16,
    scheduler = scheduler,
    progress_reporter = reporter,
    max_concurrent_trials=16
)



In [ ]:
best_config = result.get_best_config(metric="accuracy", mode="max")
print("Best Hyperparameters:", best_config)